In [4]:
#libraries 
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import docx

In [8]:
import requests
from bs4 import BeautifulSoup
import os
import mammoth
import win32com.client as win32

# URLs of the .doc/.docx files and corresponding metadata
doc_data = [
    {
        "url": "https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC001.doc",
        "metadata": '''
        <tr>
            <td colspan="2">
                <div class="title">DECISION 001</div>
                <div>DESIGNACIÓN DEL ACUERDO DE INTEGRACIÓN SUBREGIONAL CON EL NOMBRE DE ACUERDO DE CARTAGENA </div>
                <div class="metadata">Tamaño: <strong>20 Kb</strong>, Nro.Páginas: <strong>1</strong></div>
            </td>
            <td class="text-center">
                <a rel="noopener noreferrer" target="_blank" href="https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC001.doc">
                    <i class="bx bx bxs-file-doc fs-2"></i>
                </a>
            </td>
            <td>25/11/1969</td>
        </tr>
        '''
    },
    {
        "url": "https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC002.docx",
        "metadata": '''
        <tr>
            <td colspan="2">
                <div class="title">DECISION 002</div>
                <div>DELEGACIÓN DE FACULTADES DE LA COMISIÓN A LA JUNTA</div>
                <div class="metadata">Tamaño: <strong>20 Kb</strong>, Nro.Páginas: <strong>1</strong></div>
            </td>
            <td class="text-center">
                <a rel="noopener noreferrer" target="_blank" href="https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC002.doc">
                    <i class="bx bx bxs-file-doc fs-2"></i>
                </a>
            </td>
            <td>25/11/1969</td>
        </tr>
        '''
    }
]

# Function to download and extract content from .doc or .docx files
def download_and_parse_doc(url, filename):
    # Download the file
    response = requests.get(url)
    with open(filename, "wb") as file:
        file.write(response.content)
    
    # Determine file extension
    file_extension = os.path.splitext(filename)[1].lower()
    
    # Handle .doc files using win32com
    if file_extension == ".doc":
        word = win32.Dispatch("Word.Application")
        word.Visible = False
        doc = word.Documents.Open(os.path.abspath(filename))
        doc_text = doc.Content.Text
        doc.Close()
        word.Quit()
    # Handle .docx files using mammoth
    elif file_extension == ".docx":
        with open(filename, "rb") as docx_file:
            result = mammoth.extract_raw_text(docx_file)
            doc_text = result.value
    
    return doc_text

# Function to extract metadata including description from HTML snippet
def extract_metadata(html_snippet):
    soup = BeautifulSoup(html_snippet, "html.parser")
    
    title = soup.find("div", class_="title").text.strip()
    description = soup.find_all("div")[1].text.strip()  # The second <div> contains the brief description
    date = soup.find_all("td")[-1].text.strip()  # Last <td> contains the date
    pages = soup.find("div", class_="metadata").find_all("strong")[1].text.strip()  # Second <strong> contains the page number
    
    return title, description, date, pages

# Process each document and its metadata
for i, doc in enumerate(doc_data):
    # Define the filename for saving the .doc/.docx file temporarily
    filename = f"temp_doc_{i+1}{os.path.splitext(doc['url'])[1]}"
    
    # Extract content from the .doc or .docx file
    content = download_and_parse_doc(doc["url"], filename)
    
    # Extract metadata from the HTML snippet
    title, description, date, pages = extract_metadata(doc["metadata"])
    
    # Print or process the combined results
    print(f"Title: {title}")
    print(f"Description: {description}")
    print(f"Date: {date}")
    print(f"Pages: {pages}")
    print(f"Document Content: {content}\n")
    
    # Remove the temporary file
    os.remove(filename)


Title: DECISION 001
Description: DESIGNACIÓN DEL ACUERDO DE INTEGRACIÓN SUBREGIONAL CON EL NOMBRE DE ACUERDO DE CARTAGENA
Date: 25/11/1969
Pages: 1
Desígnese con el nombre de "Acuerdo de Cartagena" al Acuerdo de Integración Subregional suscrito por los Gobiernos de Bolivia, Colombia, Chile, Ecuador y Perú, en Bogotá, el 26 de mayo de 1969.



BadZipFile: File is not a zip file

In [12]:
import requests
from bs4 import BeautifulSoup
import os
import mammoth
import win32com.client as win32
from zipfile import BadZipFile
import mimetypes

# URLs of the .doc/.docx files and corresponding metadata
doc_data = [
    {
        "url": "https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC001.doc",
        "metadata": '''
        <tr>
            <td colspan="2">
                <div class="title">DECISION 001</div>
                <div>DESIGNACIÓN DEL ACUERDO DE INTEGRACIÓN SUBREGIONAL CON EL NOMBRE DE ACUERDO DE CARTAGENA </div>
                <div class="metadata">Tamaño: <strong>20 Kb</strong>, Nro.Páginas: <strong>1</strong></div>
            </td>
            <td class="text-center">
                <a rel="noopener noreferrer" target="_blank" href="https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC001.doc">
                    <i class="bx bx bxs-file-doc fs-2"></i>
                </a>
            </td>
            <td>25/11/1969</td>
        </tr>
        '''
    },
    {
        "url": "https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC002.docx",
        "metadata": '''
        <tr>
            <td colspan="2">
                <div class="title">DECISION 002</div>
                <div>DELEGACIÓN DE FACULTADES DE LA COMISIÓN A LA JUNTA</div>
                <div class="metadata">Tamaño: <strong>20 Kb</strong>, Nro.Páginas: <strong>1</strong></div>
            </td>
            <td class="text-center">
                <a rel="noopener noreferrer" target="_blank" href="https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC002.doc">
                    <i class="bx bx bxs-file-doc fs-2"></i>
                </a>
            </td>
            <td>25/11/1969</td>
        </tr>
        '''
    }
]

# Function to determine if a file is a valid .docx
def is_valid_docx(filename):
    try:
        with open(filename, "rb") as file:
            if file.read(2) == b'PK':  # Check for ZIP file signature
                return True
            return False
    except Exception as e:
        print(f"Error checking .docx file validity: {e}")
        return False

# Function to download and extract content from .doc or .docx files
def download_and_parse_doc(url, filename):
    # Download the file
    response = requests.get(url)
    with open(filename, "wb") as file:
        file.write(response.content)
    
    # Determine file extension
    file_extension = os.path.splitext(filename)[1].lower()
    
    # Handle .doc files using win32com
    if file_extension == ".doc":
        try:
            word = win32.Dispatch("Word.Application")
            word.Visible = False
            doc = word.Documents.Open(os.path.abspath(filename))
            doc_text = doc.Content.Text
            doc.Close()
            word.Quit()
        except Exception as e:
            print(f"Error processing .doc file: {e}")
            doc_text = ""
    # Handle .docx files using mammoth
    elif file_extension == ".docx":
        if is_valid_docx(filename):
            try:
                with open(filename, "rb") as docx_file:
                    result = mammoth.extract_raw_text(docx_file)
                    doc_text = result.value
            except BadZipFile as e:
                print(f"Error processing .docx file (bad zip): {e}")
                doc_text = ""
            except Exception as e:
                print(f"Error processing .docx file: {e}")
                doc_text = ""
        else:
            print(f"File {filename} is not a valid .docx file.")
            doc_text = ""
    
    return doc_text

# Function to extract metadata including description from HTML snippet
def extract_metadata(html_snippet):
    soup = BeautifulSoup(html_snippet, "html.parser")
    
    title = soup.find("div", class_="title").text.strip()
    description = soup.find_all("div")[1].text.strip()  # The second <div> contains the brief description
    date = soup.find_all("td")[-1].text.strip()  # Last <td> contains the date
    pages = soup.find("div", class_="metadata").find_all("strong")[1].text.strip()  # Second <strong> contains the page number
    
    return title, description, date, pages

# Process each document and its metadata
for i, doc in enumerate(doc_data):
    # Define the filename for saving the .doc/.docx file temporarily
    filename = f"temp_doc_{i+1}{os.path.splitext(doc['url'])[1]}"
    
    # Extract content from the .doc or .docx file
    content = download_and_parse_doc(doc["url"], filename)
    
    # Extract metadata from the HTML snippet
    title, description, date, pages = extract_metadata(doc["metadata"])
    
    # Print or process the combined results
    print(f"Title: {title}")
    print(f"Description: {description}")
    print(f"Date: {date}")
    print(f"Pages: {pages}")
    print(f"Document Content: {content}\n")
    
    # Remove the temporary file
    #s.remove(filename)



Title: DECISION 001
Description: DESIGNACIÓN DEL ACUERDO DE INTEGRACIÓN SUBREGIONAL CON EL NOMBRE DE ACUERDO DE CARTAGENA
Date: 25/11/1969
Pages: 1
Desígnese con el nombre de "Acuerdo de Cartagena" al Acuerdo de Integración Subregional suscrito por los Gobiernos de Bolivia, Colombia, Chile, Ecuador y Perú, en Bogotá, el 26 de mayo de 1969.

File temp_doc_2.docx is not a valid .docx file.
Title: DECISION 002
Description: DELEGACIÓN DE FACULTADES DE LA COMISIÓN A LA JUNTA
Date: 25/11/1969
Pages: 1
Document Content: 



In [39]:
import requests
from bs4 import BeautifulSoup
import os
import mammoth
import docx
import win32com.client as win32
from zipfile import BadZipFile

# URLs of the .doc/.docx files and corresponding metadata
doc_data = [
    {
        "url": "https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC001.doc",
        "metadata": '''
        <tr>
            <td colspan="2">
                <div class="title">DECISION 001</div>
                <div>DESIGNACIÓN DEL ACUERDO DE INTEGRACIÓN SUBREGIONAL CON EL NOMBRE DE ACUERDO DE CARTAGENA </div>
                <div class="metadata">Tamaño: <strong>20 Kb</strong>, Nro.Páginas: <strong>1</strong></div>
            </td>
            <td class="text-center">
                <a rel="noopener noreferrer" target="_blank" href="https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC001.doc">
                    <i class="bx bx bxs-file-doc fs-2"></i>
                </a>
            </td>
            <td>25/11/1969</td>
        </tr>
        '''
    },
    {
        "url": "https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC002.doc",
        "metadata": '''
        <tr>
            <td colspan="2">
                <div class="title">DECISION 002</div>
                <div>DELEGACIÓN DE FACULTADES DE LA COMISIÓN A LA JUNTA</div>
                <div class="metadata">Tamaño: <strong>20 Kb</strong>, Nro.Páginas: <strong>1</strong></div>
            </td>
            <td class="text-center">
                <a rel="noopener noreferrer" target="_blank" href="https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC002.doc">
                    <i class="bx bx bxs-file-doc fs-2"></i>
                </a>
            </td>
            <td>25/11/1969</td>
        </tr>
        '''
    },    {
        "url": "https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC003.doc",
        "metadata": '''
        <tr>
            <td colspan="2">
                <div class="title">DECISION 003</div>
                <div>APROBACIÓN DEL PLAN DE ACCIÓN PARA EL AÑO 1970</div>
                <div class="metadata">Tamaño: <strong>25 Kb</strong>, Nro.Páginas: <strong>2</strong></div>
            </td>
            <td class="text-center">
                <a rel="noopener noreferrer" target="_blank" href="https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC003.doc">
                    <i class="bx bx bxs-file-doc fs-2"></i>
                </a>
            </td>
            <td>30/12/1969</td>
        </tr>
        '''
    }
]

# Function to download and extract content from .doc or .docx files
def download_and_parse_doc(url, filename):
    # Download the file
    response = requests.get(url)
    with open(filename, "wb") as file:
        file.write(response.content)
    
    # Try to process as .docx first
    try:
        file_extension = os.path.splitext(filename)[1].lower()
        doc_text = ""
        
        if file_extension == ".docx":
            try:
                # Try using python-docx
                doc = docx.Document(filename)
                doc_text = "\n".join([para.text for para in doc.paragraphs])
            except Exception as e:
                print(f"Error using python-docx for .docx file: {e}")
                
                # Fallback to mammoth if python-docx fails
                try:
                    with open(filename, "rb") as docx_file:
                        result = mammoth.extract_raw_text(docx_file)
                        doc_text = result.value
                except BadZipFile as e:
                    print(f"Error processing .docx file (bad zip): {e}")
                except Exception as e:
                    print(f"Error processing .docx file with mammoth: {e}")
                    
        return doc_text
    
    except Exception as e:
        print(f"Error processing .docx file: {e}")
    
    # If .docx processing fails, retry with .doc
    try:
        file_extension = os.path.splitext(filename)[1].lower()
        if file_extension == ".docx":
            word = win32.Dispatch("Word.Application")
            word.Visible = False
            doc = word.Documents.Open(os.path.abspath(filename))
            doc_text = doc.Content.Text
            doc.Close()
            word.Quit()
        return doc_text
    except Exception as e:
        print(f"Error processing .doc file: {e}")
        return ""

# Function to extract metadata including description from HTML snippet
def extract_metadata(html_snippet):
    soup = BeautifulSoup(html_snippet, "html.parser")
    
    title = soup.find("div", class_="title").text.strip()
    description = soup.find_all("div")[1].text.strip()  # The second <div> contains the brief description
    date = soup.find_all("td")[-1].text.strip()  # Last <td> contains the date
    pages = soup.find("div", class_="metadata").find_all("strong")[1].text.strip()  # Second <strong> contains the page number
    
    return title, description, date, pages

# Process each document and its metadata
for i, doc in enumerate(doc_data):
    # Define the filename for saving the .doc/.docx file temporarily
    filename = f"temp_doc_{i+1}{os.path.splitext(doc['url'])[1]}"
    
    # Extract content from the .doc or .docx file
    content = download_and_parse_doc(doc["url"], filename)
    
    # Extract metadata from the HTML snippet
    title, description, date, pages = extract_metadata(doc["metadata"])
    
    # Print or process the combined results
    print(f"Title: {title}")
    print(f"Description: {description}")
    print(f"Date: {date}")
    print(f"Pages: {pages}")
    print(f"Document Content: {content}\n")
    
    # Remove the temporary file
    #os.remove(filename)


Title: DECISION 001
Description: DESIGNACIÓN DEL ACUERDO DE INTEGRACIÓN SUBREGIONAL CON EL NOMBRE DE ACUERDO DE CARTAGENA
Date: 25/11/1969
Pages: 1
Document Content: 

Title: DECISION 002
Description: DELEGACIÓN DE FACULTADES DE LA COMISIÓN A LA JUNTA
Date: 25/11/1969
Pages: 1
Document Content: 

Title: DECISION 003
Description: APROBACIÓN DEL PLAN DE ACCIÓN PARA EL AÑO 1970
Date: 30/12/1969
Pages: 2
Document Content: 



In [42]:
import requests
import os
import win32com.client as win32
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# URLs of the .doc files and corresponding metadata
doc_data = [
    {
        "source_url": "https://www.comunidadandina.org/normativa-andina/documentos-estadisticos/",
        "doc_url": "https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC001.doc",
        "metadata": '''
        <tr>
            <td colspan="2">
                <div class="title">DECISIÓN 001</div>
                <div>DESIGNACIÓN DEL ACUERDO DE INTEGRACIÓN SUBREGIONAL CON EL NOMBRE DE ACUERDO DE CARTAGENA</div>
                <div class="metadata">Tamaño: <strong>20 Kb</strong>, Nro.Páginas: <strong>1</strong></div>
            </td>
            <td class="text-center">
                <a rel="noopener noreferrer" target="_blank" href="https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC001.doc">
                    <i class="bx bx bxs-file-doc fs-2"></i>
                </a>
            </td>
            <td>25/11/1969</td>
        </tr>
        '''
    },
    {
        "source_url": "https://www.comunidadandina.org/normativa-andina/documentos-estadisticos/",
        "doc_url": "https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC002.doc",
        "metadata": '''
        <tr>
            <td colspan="2">
                <div class="title">DECISIÓN 002</div>
                <div>DELEGACIÓN DE FACULTADES DE LA COMISIÓN A LA JUNTA</div>
                <div class="metadata">Tamaño: <strong>20 Kb</strong>, Nro.Páginas: <strong>1</strong></div>
            </td>
            <td class="text-center">
                <a rel="noopener noreferrer" target="_blank" href="https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC002.doc">
                    <i class="bx bx bxs-file-doc fs-2"></i>
                </a>
            </td>
            <td>25/11/1969</td>
        </tr>
        '''
    },
        {
        "source_url": "https://www.comunidadandina.org/normativa-andina/documentos-estadisticos/",
        "doc_url": "https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC003.doc",
        "metadata": '''
        <tr>
            <td colspan="2">
                <div class="title">DECISION 003</div>
                <div>APROBACIÓN DEL PLAN DE ACCIÓN PARA EL AÑO 1970</div>
                <div class="metadata">Tamaño: <strong>25 Kb</strong>, Nro.Páginas: <strong>2</strong></div>
            </td>
            <td class="text-center">
                <a rel="noopener noreferrer" target="_blank" href="https://www.comunidadandina.org/DocOficialesFiles/decisiones/DEC003.doc">
                    <i class="bx bx bxs-file-doc fs-2"></i>
                </a>
            </td>
            <td>30/12/1969</td>
        </tr>
        '''
    }
]

# Function to download and extract content from .doc files
def download_and_parse_doc(url, filename):
    # Download the file
    response = requests.get(url)
    with open(filename, "wb") as file:
        file.write(response.content)
    
    # Process as .doc file
    try:
        word = win32.Dispatch("Word.Application")
        word.Visible = False
        doc = word.Documents.Open(os.path.abspath(filename))
        doc_text = doc.Content.Text
        doc.Close()
        word.Quit()
        return doc_text
    except Exception as e:
        print(f"Error processing .doc file: {e}")
        return ""

# Function to extract metadata including description from HTML snippet
def extract_metadata(html_snippet):
    soup = BeautifulSoup(html_snippet, "html.parser")
    title = soup.find("div", class_="title").text.strip()
    description = soup.find_all("div")[1].text.strip()  # Second <div> for description
    date = soup.find_all("td")[-1].text.strip()  # Last <td> for date
    pages = soup.find("div", class_="metadata").find_all("strong")[1].text.strip()  # Second <strong> for page number
    return title, description, date, pages

# Create an empty list to store document data
data = []

# Get today's date for date_scraped
today_date = datetime.now().strftime("%Y-%m-%d")

# Process each document and its metadata
for i, doc in enumerate(doc_data):
    # Define the filename for saving the .doc file temporarily
    filename = f"temp_doc_{i+1}.doc"
    
    # Extract content from the .doc file
    content = download_and_parse_doc(doc["doc_url"], filename)
    
    # Extract metadata from the HTML snippet
    title, description, date, pages = extract_metadata(doc["metadata"])
    
    # Append the results to the list
    data.append({
        "ID": i + 1,
        "Source_URL": doc["source_url"],
        "Document_URL": doc["doc_url"],
        "Fecha_de_Extracción": today_date,
        "Título": title,
        "Descripción": description,
        "Fecha_de_Publicación": date,
        "Páginas": pages,
        "Contenido_del_Documento": content
    })
    
    # Remove the temporary file
    os.remove(filename)

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)

# Save the DataFrame to a JSON file
df.to_json("comunidad_andina_test.json", orient="records", lines=True, force_ascii=False)

print("Data has been saved to document_data.json")


Data has been saved to document_data.json


In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from urllib.parse import urljoin

# Define the URL
url = 'https://www.comunidadandina.org/normativa-andina/decisiones/'

# Send a request to fetch the HTML content
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Extract the document table
table = soup.find('table')

# Check if table is found
if not table:
    print("Table not found.")
else:
    print("Table found.")

rows = table.find_all('tr')

# Initialize lists to store the extracted data
titles = []
descriptions = []
file_links = []
dates = []

# Process each row in the table
for row in rows[1:]:  # Skip the header row
    cells = row.find_all('td')
    
    # Debug: Print the cells
    print(f"Cells: {[cell.get_text(strip=True) for cell in cells]}")
    
    if len(cells) < 3:
        print("Skipping row due to insufficient columns.")
        continue  # Skip rows that don't have enough columns
    
    # Extract the title and description
    title_div = cells[0].find('div', class_='title')
    description_divs = cells[0].find_all('div')
    
    title = title_div.get_text(strip=True) if title_div else 'N/A'
    description = description_divs[1].get_text(strip=True) if len(description_divs) > 1 else 'N/A'
    
    # Extract metadata
    metadata_div = cells[0].find('div', class_='metadata')
    size_match = re.search(r'Tamaño:\s*<strong>(.*?)</strong>', str(metadata_div)) if metadata_div else None
    pages_match = re.search(r'Nro.Páginas:\s*<strong>(.*?)</strong>', str(metadata_div)) if metadata_div else None
    size = size_match.group(1) if size_match else 'N/A'
    pages = pages_match.group(1) if pages_match else 'N/A'
    
    # Extract the file link and date
    file_link_tag = cells[1].find('a')
    file_link = urljoin(url, file_link_tag['href']) if file_link_tag else 'N/A'
    date = cells[2].get_text(strip=True) if len(cells) > 2 else 'N/A'
    
    # Debug: Print extracted information
    print(f"Title: {title}, Description: {description}, File Link: {file_link}, Date: {date}")
    
    titles.append(title)
    descriptions.append(description)
    file_links.append(file_link)
    dates.append(date)

# Create a DataFrame
df = pd.DataFrame({
    'Title': titles,
    'Description': descriptions,
    'File Link': file_links,
    'Publication Date': dates
})

# Display the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv('comunidad_andina_documents.csv', index=False)


Table found.
Cells: ['Sin información']
Skipping row due to insufficient columns.
Empty DataFrame
Columns: [Title, Description, File Link, Publication Date]
Index: []


In [35]:
# Inspect the table rows
for row in rows:
    cells = row.find_all('td')
    
    # Debug: Print the cells
    print(f"Cells: {[cell.get_text(strip=True) for cell in cells]}")
    
    if len(cells) < 3:
        continue  # Skip rows that don't have enough columns

    # Adjust extraction logic based on actual content
    # Extract title, description, and other info based on the structure you find
    title = cells[0].get_text(strip=True)
    description = cells[1].get_text(strip=True) if len(cells) > 1 else 'N/A'
    file_link = cells[2].find('a')['href'] if len(cells) > 2 and cells[2].find('a') else 'N/A'
    date = cells[3].get_text(strip=True) if len(cells) > 3 else 'N/A'

    # Print extracted information
    print(f"Title: {title}, Description: {description}, File Link: {file_link}, Date: {date}")

    titles.append(title)
    descriptions.append(description)
    file_links.append(file_link)
    dates.append(date)



Cells: []
Cells: ['Sin información']


In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd

# Initialize the WebDriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# Open the webpage
driver.get('https://www.comunidadandina.org/normativa-andina/decisiones/')

# Extract table content
table = driver.find_element(By.TAG_NAME, 'table')
rows = table.find_elements(By.TAG_NAME, 'tr')

# Initialize lists to store the extracted data
titles = []
descriptions = []
file_links = []
dates = []

# Process each row in the table
for row in rows[1:]:  # Skip the header row
    cells = row.find_elements(By.TAG_NAME, 'td')
    
    if len(cells) < 3:
        continue  # Skip rows that don't have enough columns
    
    # Extract and print data
    title = cells[0].text
    description = cells[1].text if len(cells) > 1 else 'N/A'
    file_link = cells[2].find_element(By.TAG_NAME, 'a').get_attribute('href') if len(cells) > 2 and cells[2].find_element(By.TAG_NAME, 'a') else 'N/A'
    date = cells[3].text if len(cells) > 3 else 'N/A'
    
    print(f"Title: {title}, Description: {description}, File Link: {file_link}, Date: {date}")

    titles.append(title)
    descriptions.append(description)
    file_links.append(file_link)
    dates.append(date)

# Create and save DataFrame
df = pd.DataFrame({
    'Title': titles,
    'Description': descriptions,
    'File Link': file_links,
    'Publication Date': dates
})
df.to_csv('comunidad_andina_documents.csv', index=False)

# Close the browser
driver.quit()


ModuleNotFoundError: No module named 'selenium'

In [38]:
pip install selenium



SyntaxError: invalid syntax (4174414626.py, line 1)